### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv("NASA_API_KEY")

### CME Data

In [2]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for CME
cme_query_url = f"{base_url}{CME}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [3]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_query_url)
print(cme_response)

<Response [200]>


In [4]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [5]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(cme_json[0], indent=4))


{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [6]:
# Convert cme_json to a Pandas DataFrame  
cme = pd.DataFrame(cme_json)
cme.head()

,activityID,catalog,startTime,instruments,sourceLocation,activeRegionNum,note,submissionTime,versionId,link,cmeAnalyses,linkedEvents
0,2013-05-01T03:12:00-CME-001,M2M_CATALOG,2013-05-01T03:12Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:54Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,M2M_CATALOG,2013-05-02T05:24Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:56Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
2,2013-05-02T14:36:00-CME-001,M2M_CATALOG,2013-05-02T14:36Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T16:57Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
3,2013-05-03T18:00:00-CME-001,M2M_CATALOG,2013-05-03T18:00Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-07T17:01Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",None
4,2013-05-03T22:36:00-CME-001,M2M_CATALOG,2013-05-03T22:36Z,"[{'displayName': 'SOHO: LASCO/C2'}, {'displayN...",,NaN,,2013-08-08T13:40Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2013-05...",[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [7]:
# Keep only the activityID, startTime, linkedEvents columns

cme = cme[['activityID', 'startTime', 'linkedEvents']]
cme.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [8]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme = cme.dropna(subset=['linkedEvents'])
cme.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [9]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme.index:
      
    activityID       = cme.loc[i, 'activityID']    # Get the corresponding value from 'activityID'
    startTime        = cme.loc[i, 'startTime']     # Get the corresponding value from 'startTime'    
    linkedEvents     = cme.loc[i, 'linkedEvents']  # Get the list of dictionaries in 'linkedEvents'

    try:

        # Iterate over each dictionary in the list
        for item in linkedEvents:
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
         expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': item})
    except:
        print('missing activity')
        
# Create a new DataFrame from the expanded rows
cme_for_loop = pd.DataFrame(expanded_rows)
cme_for_loop.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [10]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

def extract_activityID_from_dict(input_dict):

    try:
        # Attempt to extract the 'activityID' from the input dictionary
        return input_dict['activityID']

    except (ValueError, TypeError) as e:
        # Handle the case where 'activityID' is not found or input is not a dict
        return None  

        # Log the error or print it for debugging

        return

extract_activityID_from_dict(cme_for_loop.loc[0,'linkedEvents'])


'2013-05-04T04:52:00-IPS-001'

In [11]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme['GST_ActivityID'] = cme['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None)
cme.loc[:, 'GST_ActivityID'] = cme['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None)
print(cme[['linkedEvents', 'GST_ActivityID']])

                                           linkedEvents GST_ActivityID
0       [{'activityID': '2013-05-04T04:52:00-IPS-001'}]           None
4       [{'activityID': '2013-05-07T04:37:00-IPS-001'}]           None
7       [{'activityID': '2013-05-12T23:30:00-IPS-001'}]           None
10    [{'activityID': '2013-05-13T01:53:00-FLR-001'}...           None
13    [{'activityID': '2013-05-13T15:40:00-FLR-001'}...           None
...                                                 ...            ...
5496    [{'activityID': '2024-04-26T00:17:00-IPS-001'}]           None
5499    [{'activityID': '2024-04-24T14:02:00-FLR-001'}]           None
5504    [{'activityID': '2024-04-25T17:03:00-FLR-001'}]           None
5516    [{'activityID': '2024-04-30T00:46:00-FLR-001'}]           None
5523    [{'activityID': '2024-05-01T22:26:00-FLR-001'}]           None

[1023 rows x 2 columns]


In [12]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme = cme.dropna(subset=['GST_ActivityID'])
print(cme[['linkedEvents', 'GST_ActivityID']])

Empty DataFrame
Columns: [linkedEvents, GST_ActivityID]
Index: []


In [13]:
# print out the datatype of each column in this DataFrame:
print(cme.dtypes)


activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object


In [15]:
# Convert the 'GST_ActivityID' column to string format 
cme['GST_ActivityID'] = cme['GST_ActivityID'].astype(str)


# Convert startTime to datetime format  
cme['startTime'] = pd.to_datetime(cme['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
cme.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
cme.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
# 1. Check column names
print("Column names:", cme.columns)

# 2. Check data types
print("Data types:\n", cme.dtypes)

# 3. Check if 'linkedEvents' column is dropped
if 'linkedEvents' not in cme.columns:
    print("'linkedEvents' column has been dropped successfully.")
else:
    print("'linkedEvents' column is still present.")


Column names: Index(['cmeID', 'startTime_CME', 'GST_ActivityID'], dtype='object')
Data types:
 cmeID                     object
startTime_CME     datetime64[ns]
GST_ActivityID            object
dtype: object
'linkedEvents' column has been dropped successfully.


In [16]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme = cme[cme['GST_ActivityID'].str.contains('GST', na=False)]
print(cme)

Empty DataFrame
Columns: [cmeID, startTime_CME, GST_ActivityID]
Index: []


### GST Data

In [17]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for GST
gst_query_url = f"{base_url}{GST}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [18]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_query_url)
print(gst_response)

<Response [200]>


In [19]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

In [20]:
# Convert the response variable to json and store it as a variable named gst_json
# Format the first result in JSON with indentation for readability
print(json.dumps(gst_json[0], indent=4))

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [21]:
# Convert gst_json to a Pandas DataFrame  
gst = pd.DataFrame(gst_json)
gst.head()

,gstID,startTime,allKpIndex,link,linkedEvents,submissionTime,versionId
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,"[{'observedTime': '2013-06-01T01:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,[{'activityID': '2013-05-31T15:45:00-HSS-001'}],2013-07-15T19:26Z,1
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,"[{'observedTime': '2013-06-07T03:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,[{'activityID': '2013-06-02T20:24:00-CME-001'}],2013-07-15T19:41Z,1
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,"[{'observedTime': '2013-06-29T03:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,None,2013-09-25T04:48Z,2
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,"[{'observedTime': '2013-10-02T06:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,[{'activityID': '2013-09-29T22:40:00-CME-001'}...,2013-10-02T13:23Z,1
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,"[{'observedTime': '2013-12-08T03:00Z', 'kpInde...",https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,[{'activityID': '2013-12-04T23:12:00-CME-001'}...,2013-12-08T03:42Z,1


In [22]:
# Keep only the columns: activityID, startTime, linkedEvents
gst = gst[['gstID', 'startTime', 'linkedEvents']]
gst.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [23]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst = gst.dropna(subset=['linkedEvents'])
gst.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [24]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_exploded = gst.explode('linkedEvents')

# Reset the index
gst_exploded.reset_index(drop=True, inplace=True)

# Drop missing values
gst_exploded.dropna(subset=['linkedEvents'], inplace=True)

# Display the resulting DataFrame
print(gst_exploded)

                           gstID          startTime  \
0    2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1    2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
..                           ...                ...   
200  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
201  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
202  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
203  2024-04-19T18:00:00-GST-001  2024-04-19T18:00Z   
204  2024-04-19T18:00:00-GST-001  2024-04-19T18:00Z   

                                      linkedEvents  
0    {'activityID': '2013-05-31T15:45:00-HSS-001'}  
1    {'activityID': '2013-06-02T20:24:00-CME-001'}  
2    {'activityID': '2013-09-29T22:40:00-CME-001'}  
3    {'activityID': '2013-10-02T01:54:00-IPS-001'}  
4    {'activityID': '2013-10-02T02:47:00-MPC-001'}  
..                   

In [25]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict['activityID']  # Adjust this according to your dictionary structure
    except (ValueError, TypeError) as e:
        return None  # Return None if there's an error

gst['CME_ActivityID'] = gst['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
print(gst)


# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst.dropna(subset=['CME_ActivityID'], inplace=True)
print(gst)

                           gstID          startTime  \
0    2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1    2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
3    2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4    2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   
5    2014-02-19T03:00:00-GST-001  2014-02-19T03:00Z   
..                           ...                ...   
112  2023-12-18T06:00:00-GST-001  2023-12-18T06:00Z   
113  2024-03-03T18:00:00-GST-001  2024-03-03T18:00Z   
114  2024-03-23T21:00:00-GST-001  2024-03-23T21:00Z   
115  2024-03-24T12:00:00-GST-001  2024-03-24T12:00Z   
116  2024-04-19T18:00:00-GST-001  2024-04-19T18:00Z   

                                          linkedEvents CME_ActivityID  
0      [{'activityID': '2013-05-31T15:45:00-HSS-001'}]           None  
1      [{'activityID': '2013-06-02T20:24:00-CME-001'}]           None  
3    [{'activityID': '2013-09-29T22:40:00-CME-001'}...           None  
4    [{'activityID': '2013-12-04T23:12:00-CME-001'}

In [26]:
# Convert the 'gstID' column to string format 
gst['gstID'] = gst['gstID'].astype(str)


# Convert startTime to datetime format  
gst['startTime'] = pd.to_datetime(gst['startTime'])


# Rename startTime to startTime_GST 
gst.rename(columns={'startTime': 'startTime_GST'}, inplace=True)


# Drop linkedEvents
gst.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
# 1. Check column names
print("Column names:", gst.columns)

# 2. Check data types
print("Data types:\n", gst.dtypes)

# 3. Check if 'linkedEvents' column is dropped
if 'linkedEvents' not in gst.columns:
    print("'linkedEvents' column has been dropped successfully.")
else:
    print("'linkedEvents' column is still present.")

Column names: Index(['gstID', 'startTime_GST', 'CME_ActivityID'], dtype='object')
Data types:
 gstID                     object
startTime_GST     datetime64[ns]
CME_ActivityID            object
dtype: object
'linkedEvents' column has been dropped successfully.


In [27]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst = gst[gst['CME_ActivityID'].str.contains('CME', na=False)]
print(gst)

Empty DataFrame
Columns: [gstID, startTime_GST, CME_ActivityID]
Index: []


### Merge both datatsets

In [28]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst, cme, 
                     left_on=['gstID', 'CME_ActivityID'], 
                     right_on=['GST_ActivityID', 'cmeID'], 
                     how='outer')

print(merged_df.shape)

(0, 6)


In [29]:
# Verify that the new DataFrame has the same number of rows as cme and gst
# Check the number of rows in the original DataFrames
cme_rows = cme.shape[0]
gst_rows = gst.shape[0]

# Check the number of rows in the merged DataFrame
merged_rows = merged_df.shape[0]

# Print the results
print(f"Number of rows in cme: {cme_rows}")
print(f"Number of rows in gst: {gst_rows}")
print(f"Number of rows in merged DataFrame: {merged_rows}")

# Verify if the number of rows in the merged DataFrame matches the original DataFrames
if merged_rows == cme_rows + gst_rows:
    print("The merged DataFrame has the expected number of rows.")
else:
    print("The merged DataFrame does not have the expected number of rows.")

Number of rows in cme: 0
Number of rows in gst: 0
Number of rows in merged DataFrame: 0
The merged DataFrame has the expected number of rows.


### Computing the time it takes for a CME to cause a GST

In [32]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['startTime_gst'] = pd.to_datetime(merged_df['startTime_GST'])
merged_df['startTime_cme'] = pd.to_datetime(merged_df['startTime_CME'])

merged_df['timeDiff'] = merged_df['startTime_gst'] - merged_df['startTime_cme']
print(merged_df[['startTime_GST', 'startTime_CME', 'timeDiff']].head())


Empty DataFrame
Columns: [startTime_GST, startTime_CME, timeDiff]
Index: []


In [33]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
# Use describe() to compute summary statistics
time_diff_stats = merged_df['timeDiff'].describe()

# To specifically get the mean and median
mean_time_diff = time_diff_stats['mean']
median_time_diff = time_diff_stats['50%']  # The 50th percentile is the median

print(f"Mean time difference: {mean_time_diff}")
print(f"Median time difference: {median_time_diff}")

Mean time difference: NaT
Median time difference: NaT


### Exporting data in csv format

In [34]:
# Export data to CSV without the index
merged_df.to_csv("output_file.csv", index=False)